In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
MIMICtable = pd.read_csv("/data/localhost/taufiq/export-dir/MIMICtable-1hourly.csv")
MIMICtable['bloc'] = MIMICtable.bloc.astype(int)
MIMICtable['icustay_id'] = MIMICtable.icustay_id.astype(int)

In [3]:
MIMICtable.head()

,bloc,icustay_id,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,mechvent,Shock_Index,PaO2_FiO2,median_dose_vaso,max_dose_vaso,input_total,input_1hourly,output_total,output_1hourly,cumulated_balance
0,1,3,7.245400e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.663900,599.999991,0.0,0.0,6267.0,0.0,9490.0,4705.0,-3223.0
1,2,3,7.245403e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.672340,339.999995,0.0,0.0,6277.0,10.0,14095.0,4605.0,-7818.0
2,3,3,7.245407e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.649789,339.999995,0.0,0.0,6297.0,20.0,18540.0,4445.0,-12243.0
3,4,3,7.245410e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.634454,339.999995,0.0,0.0,6317.0,20.0,22845.0,4305.0,-16528.0
4,5,3,7.245414e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.645714,497.499993,0.0,0.0,6337.0,20.0,27050.0,4205.0,-20713.0


In [4]:
MIMICtable['Potassium'] = MIMICtable['Potassium']*39.0983
MIMICtable['Sodium'] = MIMICtable['Sodium']*22.989769
MIMICtable['Chloride'] = MIMICtable['Chloride']*35.45
MIMICtable['Glucose'] = MIMICtable['Glucose']*10
MIMICtable['Creatinine'] = MIMICtable['Creatinine']*10
MIMICtable['Calcium'] = MIMICtable['Calcium']*10
MIMICtable['Albumin'] = MIMICtable['Albumin']*10**4
MIMICtable['HCO3'] = MIMICtable['HCO3']*61.0168
MIMICtable['Arterial_lactate'] = MIMICtable['Arterial_lactate']*90.08
MIMICtable['WBC_count'] = MIMICtable['WBC_count']*10**3

In [5]:
icustayids = set(MIMICtable['icustay_id'])

In [6]:
len(icustayids)

17161

In [34]:
trajectory_length = 5
MIMICtable_filtered = MIMICtable[MIMICtable['bloc']<=trajectory_length]

In [35]:
len(MIMICtable_filtered)

53852

In [36]:
len(MIMICtable)

610551

In [88]:
processed_data = pd.DataFrame()
filtered_icustayids = []
for icustay_id in tqdm(icustayids):
    data = MIMICtable_filtered[MIMICtable_filtered['icustay_id'] == icustay_id]
    if len(data) > 0 and set(data['bloc']) == set(range(1,len(data)+1)):
        filtered_icustayids.append(icustay_id)
        rows = pd.DataFrame([[-1]*len(data.columns)]*(trajectory_length-len(data)), columns=data.columns)
        data = data.append(rows, ignore_index=True)
        data['icustay_id'] = icustay_id
        processed_data = processed_data.append(data, ignore_index=True)

100%|██████████| 17161/17161 [01:49<00:00, 156.80it/s]


In [37]:
# Only extract trajectories of length two
processed_data = pd.DataFrame()
filtered_icustayids = []
for icustay_id in tqdm(icustayids):
    data = MIMICtable_filtered[MIMICtable_filtered['icustay_id'] == icustay_id]
    if len(data) > 0 and set(data['bloc']) == set(range(1,trajectory_length+1)):
        filtered_icustayids.append(icustay_id)
        processed_data = processed_data.append(data, ignore_index=True)

100%|██████████| 17161/17161 [00:30<00:00, 561.67it/s]


In [38]:
len(processed_data)/5

6177.0

In [21]:
processed_data.head()
processed_data['input_total'] = processed_data['input_total'] - processed_data['input_1hourly']

In [22]:
processed_data.to_csv(f"/data/localhost/taufiq/export-dir/MIMIC-1hourly-length-{trajectory_length}.csv", index=False)